## Start

In [1]:
import json
import os
import torch
import tqdm
from shutil import copyfile

# Detectron imports
from detectron2.engine import launch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.data import build_detection_test_loader, MetadataCatalog

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from train_utils import ActiveTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml'
args.random_seed = 5000
args.resume=True
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50162', eval_only=False, inference_config='/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=5000, resume=True, test_dataset='bdd_val')


In [3]:
# run this once per session only
cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

Loading config /home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.


[01/29 18:33:19 detectron2]: Rank of current process: 0. World size: 1
[01/29 18:33:20 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.7.1 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0,1,2,3             Tesla V100-SXM2-32GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow                  8.1.0
torchvision             0.8.2 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torchvision
torchvision arch flags  3.5, 5.0

[01/29 18:33:20 detectron2]: Full config saved to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_5000/config.yaml


In [4]:
#cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

# Make sure only 1 data point is processed at a time. This simulates
# deployment.
cfg.defrost()
cfg.DATALOADER.NUM_WORKERS = 32
cfg.SOLVER.IMS_PER_BATCH = 1

cfg.MODEL.DEVICE = device.type

# Set up number of cpu threads
torch.set_num_threads(cfg.DATALOADER.NUM_WORKERS)

# Create inference output directory and copy inference config file to keep
# track of experimental settings
inference_output_dir = os.path.join(
    cfg['OUTPUT_DIR'],
    'inference',
    args.test_dataset,
    os.path.split(args.inference_config)[-1][:-5])
os.makedirs(inference_output_dir, exist_ok=True)
copyfile(args.inference_config, os.path.join(
    inference_output_dir, os.path.split(args.inference_config)[-1]))

# Get category mapping dictionary:
train_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
test_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    args.test_dataset).thing_dataset_id_to_contiguous_id

# If both dicts are equal or if we are performing out of distribution
# detection, just flip the test dict.
if (train_thing_dataset_id_to_contiguous_id == test_thing_dataset_id_to_contiguous_id) or (
        cfg.DATASETS.TRAIN[0] == 'coco_not_in_voc_2017_train'):
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
else:
    # If not equal, two situations: 1) BDD to KITTI and 2) COCO to PASCAL
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
    if 'voc' in args.test_dataset and 'coco' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.COCO_TO_VOC_CONTIGUOUS_ID.items())
    elif 'kitti' in args.test_dataset and 'bdd' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.BDD_TO_KITTI_CONTIGUOUS_ID.items())
    else:
        ValueError(
            'Cannot generate category mapping dictionary. Please check if training and inference datasets are compatible.')
    cat_mapping_dict = dict(
        (dataset_mapping_dict[k], v) for k, v in cat_mapping_dict.items())

# Build predictor
model = build_model(cfg)

DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=True)

trainer = ActiveTrainer(cfg, model)
#trainer.resume_or_load(resume=True)
trainer.train()


predictor = build_predictor(cfg, model)



[01/29 18:33:24 fvcore.common.checkpoint]: Loading checkpoint from /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_5000/model_final.pth
[01/29 18:33:29 d2.data.datasets.coco]: Loading /public-dataset/BDD/bdd100k/labels/train_coco_format.json takes 4.11 seconds.
[01/29 18:33:29 d2.data.datasets.coco]: Loaded 69863 images in COCO format from /public-dataset/BDD/bdd100k/labels/train_coco_format.json
[01/29 18:33:31 d2.data.build]: Removed 458 images with no usable annotations. 69405 images left.
[01/29 18:33:34 d2.data.build]: Distribution of instances among all 7 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 713211       |    bus     | 11672        |   truck    | 29971        |
|   person   | 91349        |   rider    | 4517         |    bike    | 7210         |
|  

In [5]:
test_data_loader = build_detection_test_loader(
    cfg, dataset_name=args.test_dataset)
final_output_list = []
if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)
                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                pbar.update(1)
                break
    with open(os.path.join(inference_output_dir, 'coco_instances_results.json'), 'w') as fp:
        json.dump(final_output_list, fp, indent=4,
                  separators=(',', ': '))

  0%|          | 1/10000 [00:02<7:04:16,  2.55s/it]


In [6]:
outputs

Instances(num_instances=89, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[7.9015e+02, 3.5857e+02, 8.9976e+02, 4.2730e+02],
        [1.9161e-01, 3.3675e+02, 5.1247e+01, 4.0177e+02],
        [1.2036e+03, 4.0912e+02, 1.2800e+03, 5.2821e+02],
        [2.4540e+02, 3.4409e+02, 3.4920e+02, 3.9724e+02],
        [7.4842e+02, 3.5874e+02, 8.1164e+02, 4.1051e+02],
        [8.8006e+02, 3.7351e+02, 9.5733e+02, 4.4558e+02],
        [4.6440e+01, 3.4691e+02, 1.2887e+02, 3.9949e+02],
        [7.2704e+02, 3.6294e+02, 7.6089e+02, 4.0232e+02],
        [9.3366e+02, 3.3086e+02, 1.2102e+03, 4.8222e+02],
        [5.9691e+02, 3.5298e+02, 6.3488e+02, 3.8390e+02],
        [7.0329e+02, 3.6286e+02, 7.3747e+02, 3.9585e+02],
        [2.0109e+02, 3.3850e+02, 2.7974e+02, 3.8686e+02],
        [6.8106e+02, 3.5683e+02, 7.2126e+02, 3.9115e+02],
        [5.8093e+02, 3.5515e+02, 6.0138e+02, 3.7579e+02],
        [8.4282e+02, 3.6545e+02, 9.4021e+02, 4.3977e+02],
        [4.4674e+02, 3.5368e+02, 4.7564e

In [7]:
input_im

[{'file_name': '/public-dataset/BDD/bdd100k/images/100k/val/b1c66a42-6f7d68ca.jpg',
  'height': 720,
  'width': 1280,
  'image_id': 0,
  'image': tensor([[[255, 255, 255,  ...,  56,  53,  51],
           [251, 254, 255,  ...,  55,  52,  50],
           [226, 240, 250,  ...,  54,  51,  49],
           ...,
           [ 48,  48,  48,  ...,  27,  27,  27],
           [ 47,  47,  47,  ...,  25,  25,  25],
           [ 47,  47,  47,  ...,  25,  25,  25]],
  
          [[255, 255, 254,  ...,  88,  85,  83],
           [249, 252, 254,  ...,  87,  84,  82],
           [223, 238, 248,  ...,  86,  83,  81],
           ...,
           [ 57,  57,  57,  ...,  22,  22,  22],
           [ 56,  56,  56,  ...,  20,  20,  20],
           [ 56,  56,  56,  ...,  20,  20,  20]],
  
          [[225, 220, 216,  ..., 123, 120, 118],
           [215, 217, 219,  ..., 122, 119, 117],
           [191, 206, 216,  ..., 121, 118, 116],
           ...,
           [ 60,  60,  60,  ...,  19,  19,  19],
           [ 59,

In [8]:
trainer.dataset.label_randomly(100)

In [9]:
trainer.rebuild_trainer()

In [10]:
trainer.train()

[01/29 18:33:55 d2.engine.train_loop]: Starting training from iteration 0
[01/29 18:34:22 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_5000/model_final.pth
[01/29 18:34:23 d2.engine.hooks]: Overall training speed: 198 iterations in 0:00:26 (0.1342 s / it)
[01/29 18:34:23 d2.engine.hooks]: Total training time: 0:00:27 (0:00:00 on hooks)


In [11]:
test_data_loader = build_detection_test_loader(
    cfg, dataset_name=args.test_dataset)
final_output_list = []
if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)
                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                pbar.update(1)
                break
    with open(os.path.join(inference_output_dir, 'coco_instances_results.json'), 'w') as fp:
        json.dump(final_output_list, fp, indent=4,
                  separators=(',', ': '))

[01/29 18:35:36 d2.data.datasets.coco]: Loaded 10000 images in COCO format from /public-dataset/BDD/bdd100k/labels/val_coco_format.json
[01/29 18:35:37 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/29 18:35:37 d2.data.common]: Serializing 10000 elements to byte tensors and concatenating them all ...
[01/29 18:35:37 d2.data.common]: Serialized dataset takes 9.03 MiB


  0%|          | 1/10000 [00:02<7:48:47,  2.81s/it]


In [12]:
outputs

Instances(num_instances=86, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[7.9082e+02, 3.5794e+02, 8.9813e+02, 4.2818e+02],
        [4.7570e-01, 3.3647e+02, 5.0673e+01, 4.0307e+02],
        [1.2034e+03, 4.0697e+02, 1.2800e+03, 5.2959e+02],
        [2.4780e+02, 3.4404e+02, 3.4914e+02, 3.9819e+02],
        [8.8001e+02, 3.7421e+02, 9.5655e+02, 4.4569e+02],
        [7.4735e+02, 3.5858e+02, 8.1182e+02, 4.1188e+02],
        [4.6173e+01, 3.4660e+02, 1.2887e+02, 4.0050e+02],
        [7.2668e+02, 3.6286e+02, 7.6067e+02, 4.0218e+02],
        [9.3351e+02, 3.2980e+02, 1.2072e+03, 4.8558e+02],
        [5.9681e+02, 3.5381e+02, 6.3455e+02, 3.8489e+02],
        [2.0167e+02, 3.3862e+02, 2.7858e+02, 3.8811e+02],
        [6.8136e+02, 3.5682e+02, 7.2095e+02, 3.9145e+02],
        [7.0339e+02, 3.6306e+02, 7.3678e+02, 3.9602e+02],
        [5.8111e+02, 3.5600e+02, 6.0111e+02, 3.7647e+02],
        [4.4614e+02, 3.5323e+02, 4.7512e+02, 3.7296e+02],
        [5.1401e+02, 3.5425e+02, 5.3560e

In [13]:
input_im

[{'file_name': '/public-dataset/BDD/bdd100k/images/100k/val/b1c66a42-6f7d68ca.jpg',
  'height': 720,
  'width': 1280,
  'image_id': 0,
  'image': tensor([[[255, 255, 255,  ...,  56,  53,  51],
           [251, 254, 255,  ...,  55,  52,  50],
           [226, 240, 250,  ...,  54,  51,  49],
           ...,
           [ 48,  48,  48,  ...,  27,  27,  27],
           [ 47,  47,  47,  ...,  25,  25,  25],
           [ 47,  47,  47,  ...,  25,  25,  25]],
  
          [[255, 255, 254,  ...,  88,  85,  83],
           [249, 252, 254,  ...,  87,  84,  82],
           [223, 238, 248,  ...,  86,  83,  81],
           ...,
           [ 57,  57,  57,  ...,  22,  22,  22],
           [ 56,  56,  56,  ...,  20,  20,  20],
           [ 56,  56,  56,  ...,  20,  20,  20]],
  
          [[225, 220, 216,  ..., 123, 120, 118],
           [215, 217, 219,  ..., 122, 119, 117],
           [191, 206, 216,  ..., 121, 118, 116],
           ...,
           [ 60,  60,  60,  ...,  19,  19,  19],
           [ 59,

In [15]:
len(outputs.pred_boxes_covariance)

86